# Сценарии использования модели BERT из фреймворка NNTile

In [ ]:
# Preliminary setup of experimental environment
import os
from pathlib import Path
import subprocess

nntile_dir = Path.cwd() / ".."

# Set environment variables
os.environ["CUDA_VISIBLE_DEVICES"] = "4" # Limit CUDA visibility
os.environ["OMP_NUM_THREADS"] = "1" # Disable BLAS parallelism
os.environ["PYTHONPATH"] = str(nntile_dir / "build" / "wrappers" / "python") # Path to a binary dir of NNTile Python wrappers

# All StarPU environment variables are available at https://files.inria.fr/starpu/doc/html/ExecutionConfigurationThroughEnvironmentVariables.html
os.environ["STARPU_NCPU"] = "2" # Use only 1 CPU core
os.environ["STARPU_NCUDA"] = "1" # Use only 1 CUDA device
os.environ["STARPU_SILENT"] = "1" # Do not show lots of StarPU outputs
os.environ["STARPU_SCHED"] = "dmdasd" # Name StarPU scheduler to be used
os.environ["STARPU_FXT_TRACE"] = "0" # Do not generate FXT traces
os.environ["STARPU_WORKERS_NOBIND"] = "1" # Do not bind workers (it helps if several instances of StarPU run in parallel)
os.environ["STARPU_PROFILING"] = "1" # This enables logging performance of workers and bandwidth of memory nodes
os.environ["STARPU_HOME"] = str(Path.cwd() / "starpu") # Main directory in which StarPU stores its configuration files
os.environ["STARPU_PERF_MODEL_DIR"] = str(Path(os.environ["STARPU_HOME"]) / "sampling") # Main directory in which StarPU stores its performance model files
os.environ["STARPU_PERF_MODEL_HOMOGENEOUS_CPU"] = "1" # Assume all CPU cores are equal
os.environ["STARPU_PERF_MODEL_HOMOGENEOUS_CUDA"] = "1" # Assume all CUDA devices are equal
os.environ["STARPU_HOSTNAME"] = "GPT2_example" # Force the hostname to be used when managing performance model files
os.environ["STARPU_FXT_PREFIX"] = str(Path(os.environ["STARPU_HOME"]) / "fxt") # Directory to store FXT traces if enabled

In [ ]:
# Prepare TinyStories dataset into train.bin file
# Describe parameters and arguments
!python ../wrappers/python/examples/mlm_data_preparation.py --seq-len=512 --batch-size=8 --dataset-select=100

## Аргументы скрипта ```bert_training.py```, который используется для запуска всех сценариев ниже

- ```remote-model-name```, (str, default="bert-base-uncased"): имя модели на основе архитектуры Bert, которая находится в инфраструктуре Hugging Face и будет использована для инициализации конфигурации и начального состояния модели NNTile.  
- ```pretrained```, (choices=["local", "remote"], default="local"): тип источника предобученной модели. Вариант remote подразумевает загрузку модели remote-model-name из инфраструктуры Huggingface. Вариант local требует указания пути к файлу конфигурации (```config-path```) для начала обучения из случайно инициализированного состояния или для продолжения обучения, если заодно предоставлен путь к файлу контрольной точки (```checkpoint-path```).
- ```checkpoint-path```, (str, default=""): путь к сохранённому состоянию весов заранее обученной модели. Если файл доступен, обучение продолжится из этого состояния.
- ```config-path```, (str, default=""): путь к конфигурационному .json файлу, который необходимо предоставить в текущей версии, если параметр pretrained установлен на "local".  
- ```save-checkpoint-path```, (str, default=".model"): путь, в котором сохранится состояние модели в конце текущего цикла обучения. 
- ```optimizer```, (choices=["sgd", "adam", "adamw"], default="adam"): параметр определяет тип оптимизатора, который будет использован в процессе обучения; текущая версия NNTile поддерживает три различных метода оптимизации.  
- ```model-path```, (str, default=".model"): путь, где сохраняются ранее загруженные модели из удаленного источника Huggingface, облегчая доступ для дальнейшего использования.  
- ```seq-len```, (int, default=1024): длина входной последовательности токенов для обучения,
- ```batch-size```, (int, default=1): размер батча для процесса обучения, который указывает количество обрабатываемых предложений по seq-len токенов между шагами оптимизатора функции потерь. 
- ```minibatch-size```, (int, default=-1): размер батча, под который выделяется память при обучении. Весь батч разбивается на целые минибатчи. Все минибатчи из одного батча один за другим «прогоняются» через модель для накапливания градиентов параметров.
- ```minibatch-size-tile```, (type=int, default=-1): размер батча, который попадает на ЦПУ или ГПУ для вычислений. Каждый минибатч должен делиться на целое количество тайлов минибатча.
- ```hidden-size-tile```, (type=int, default=-1): размер кусочков (тайлов), на которые дробится размерность "hidden size" (также встречается под названием "embedding size") – размер многомерного пространства, в которое отображаются входящие токены. На обработку на ЦПУ и ГПУ попадают только «кусочные» тензоры, имеющие размер hidden-size-tile по соответствующей оси.
- ```intermediate-size-tile```, (type=int, default=-1): размер кусочков (тайлов), на которые дробится размерность "intermediate size". На обработку на ЦПУ и ГПУ попадают только «кусочные» тензоры, имеющие размер intermediate-size-tile по соответствующей оси.
- ```n-head-tile```, (type=int, default=-1): размер кусочков (тайлов), на которые дробится количество голов трансформерного слоя. На обработку на ЦПУ и ГПУ попадают только «кусочные» тензоры, имеющие размер n-head-tile по соответсвующей оси.
- ```dtype```, (choices=["fp32", "fp64", "fp32_fast_tf32", "bf16", "fp32_fast_fp16", "fp32_fast_bf16"], default="fp32"): установка типа данных из поддерживаемых фреймворком NNTile в текущем состоянии. Позволяет пользователям выбирать подходящий вариант в зависимости от их требований.  
- ```restrict```, (choices=["cpu", "cuda", None], default=None): опция позволяет пользователям задавать ограничения на вычислительные ресурсы, используемые во время обучения. Выбор "cpu" ограничивает обучение только ядрами ЦП, "cuda" ограничивает обучение только на GPU ядрах, в то время как установка в None позволяет использовать все из доступных вычислительных ядер,
- ```flash-attention```, (action="store_true"): логический флаг, который при задействовании в строке аргументов, включает текущую реализацию алгоритма Flash Attention (низкоуровневые ядра Flash Attention на данный момент отсутствуют) для обработки данных в «механизме внимания» нейросетей типа «Трансформеры». 
- ```use-redux```, (action="store_true"): логический флаг, который при задействовании в строке аргументов, позволяет вычислять зависящие друг от друга задачи одновременно с последующей редукцией результатов в один тензор.
- ```dataset-path```, (default=".data"): путь к директории, где сохраняются ранее подготовленные наборы данных.
- ```dataset-file```, (default=""): путь (относительно dataset-path) к .bin файлу, который создается на в блоке подготовки данных к обучению.
- ```lr```, (type=float, default=1e-4): длина шага для оптимизационного алгоритма.
- ```nepochs```, (type=int, default=1): количество полных проходов по обучающей выборке
- ```label-mask-token``` (type=int, default=3): индекс токена, который отвечает за маскирование элементов последовательности. Должен быть согласован с используемым токенизатором для избежания пересечений индексов токенов маскированных и обычных
- ```n-masked-tokens-per-seq``` (type=int, default=1): число токенов в каждой последовательности, которое будет маскировано случайным образом 
- ```n-masks-per-seq``` (type=int, default=1): число масок, которые применяются к каждой последовательности. Эти маски применяются до начала обучения и результат их применения используется на всех эпохах

## 1. Обучение из случайного начального состояния и сохранение весов обученной модели



In [ ]:
!python ../wrappers/python/examples/bert_training.py --pretrained=local \
                                                     --config-path="../wrappers/python/examples/bert_config.json" \
                                                     --save-checkpoint-path=".model/nntile_checkpoint.pt" \
                                                     --optimizer="adam" \
                                                     --lr=1e-5 --dtype=fp32_fast_fp16 --nepochs=1 \
                                                     --batch-size=8 --minibatch-size=4 --n-masks-per-seq=3 \
                                                     --seq-len=1024 --dataset-file="tinystories/train.bin"

## 2. Загрузка весов модели из контрольной точки и продолжение обучения с другим типом данных.

Для этого снова требуется установить параметр pretrained в значение local, параметр config-path должен указывать на ранее созданный файл конфигурации в формате .json, а также checkpoint-path должен указывать на существующий файл контрольной точки в формате PyTorch. Обучение может быть продолжено с использованием другого типа данных и на другом наборе вычислительных узлов. Например, здесь мы переключаемся на тип  типа fp32_fast_tf32.

In [ ]:
!python ../wrappers/python/examples/bert_training.py --pretrained=local \
                                                     --checkpoint-path=".model/nntile_checkpoint.pt" \
                                                     --config-path="../wrappers/python/examples/bert_config.json" \
                                                     --save-checkpoint-path=".model/nntile_further_checkpoint.pt" \
                                                     --optimizer="adam" --lr=1e-5 --dtype=fp32_fast_tf32 \
                                                     --nepochs=1 --batch-size=8 --minibatch-size=4 \
                                                     --dataset-file="tinystories/train.bin" --n-masks-per-seq=3

## 3. Продолжение обучения модели, загруженной из инфраструктуры Hugging Face. 

Фреймворк NNTile в настоящее время поддерживает продолжение обучения модели, загруженной из удаленного источника, в нашем примере - из библиотеки инфраструктуры Hugging Face. 
Веса загруженной модели передаются в модель, реализованную в NNTile. 
Для запуска такого сценария необходимо установить параметр pretrained в значение remote. 
Параметры config-path и checkpoint-path больше не требуются, так как конфигурация модели и веса слоев будут получены из загруженной модели.
Обучение может быть продолжено с использованием любого типа данных и на любом наборе вычислительных узлов, которые поддерживают выбранный тип данных.
В примере ниже здесь мы переключаемся на тип BF16.

In [ ]:
!python ../wrappers/python/examples/bert_training.py --restrict="cuda" --pretrained=remote \
                                                     --save-checkpoint-path=".model/nntile_remote_checkpoint.pt" \
                                                     --optimizer="adam" --lr=1e-13 --dtype=bf16 --nepochs=1 \
                                                     --batch-size=8 --minibatch-size=4 --n-masks-per-seq=1 --seq-len=1024 \
                                                     --dataset-file="tinystories/train.bin"
